In [1]:
import getpass
import heapq
import time
from binance.client import Client
api_key = 'hnuw9lrplsmsm3WR1ryAGfbDPaxwR5BbwMjRgbCppEu9PzGjtghoQTEnFscT2SUi'
api_secret = getpass.getpass('api_secret:')

api_secret:········


In [33]:
""" Define Functions """
client = Client(api_key, api_secret)

def calc_gain(t,price_dict):
    
    """ Calculate price difference version 1/19/2018"""
    
    # Compute prices for triangle
    price1 = float(price_dict[t[0]])
    if t[0][0:len(root)] != root:
        price1 = 1/price1

    price2 = float(price_dict[t[1]])
    m1 = t[0].replace(root,"")
    if t[1][0:len(m1)] != m1:
        price2 = 1/price2

    price3 = float(price_dict[t[2]])
    m2 = t[2].replace(root,"")
    if t[2][0:len(m2)] != m2:
        price3 = 1/price3

    price_final = (price1 * price2 * price3)
    return price_final

def price_api():
    
    """ API Request """
    
    # Get all symbol prices
    prices = client.get_all_tickers()
    
    # Organize data
    symbols = []
    price_dict = {}
    for p in prices:
        symbols.append(p['symbol'])
        price_dict.update({p['symbol']:p['price']})
    return price_dict, symbols
    
def get_trades(root, price_dict, symbols):
    

    """ Get trades version 1/19/2018 """
    
    # Explore medium 1
    md1 = [ k for k in symbols if k.find(root) != -1]

    # Explore medium 2
    md2p = {}
    for m in md1:
        n = m.replace(root,"")
        md2t = [ k for k in symbols if k.find(n) != -1 and k.find(root) == -1]
        md2p.update({n:md2t})

    # Validate triangle
    md2 = {}
    for m in md2p:
        n = md2p[m]
        n2 = []
        for item in n:
            n2.append(item.replace(m,""))
        for item in n2:
            vset = [ k for k in symbols if k.find(item) != -1 and k.find(root) != -1]
            if len(vset) == 1:
                md2.update({m:vset})

    # Format triangle
    triangle = []
    for m in md2:
        first_trade = [ k for k in symbols if k.find(m) != -1 and k.find(root) != -1][0]
        last_trade = md2[m][0]
        medium2 = last_trade.replace(root,"")
        middle_trade = [ k for k in symbols if k.find(m) != -1 and k.find(medium2) != -1][0]

        """This excludes VIB and VIBE currencies. Implemented for VIBETH and VIBE confusion."""
        if first_trade.find("VIB") == -1 and middle_trade.find("VIB") == -1:
            triangle.append([first_trade,middle_trade,last_trade])

    # Compute gains
    gains = {}
    for t in triangle:
        
        price_final = abs(1-(calc_gain(t,price_dict)))
        gains.update({str(t):price_final})

    # Find top n trades
    topn = heapq.nlargest(10, gains, key=gains.get)
    
    trades = {}
    counter = 1
    # Find trade direction
    for t in topn:
    
        t = t.replace("[","")
        t = t.replace("]","")
        t = t.replace("'","")
        t = t.replace(" ","")
        t = t.split(",")
        price_final = calc_gain(t,price_dict)
        #print(t)
        #print(price_final)

        if price_final < 1:
            temp = []
            temp.append(t[2])
            temp.append(t[1])
            temp.append(t[0])
            t = temp
            price_final = calc_gain(t,price_dict)
            #print("Flip")
        price_abs = abs(price_final-1)
        #print(t)
        #print(price_abs)
        trades.update({counter:{"name":t,"gain":price_abs}})
        counter = counter + 1
    return trades



In [51]:
""" Define Functions """

def calc_depth(root, t, elimit):

    price_dict2 = {}
    price_dict3 = {}

    # Compute prices for triangle
    price1 = float(price_dict[t[0]])
    if t[0][0:len(root)] != root:
        price1 = 1/price1

    price2 = float(price_dict[t[1]])
    m1 = t[0].replace(root,"")
    if t[1][0:len(m1)] != m1:
        price2 = 1/price2

    price3 = float(price_dict[t[2]])
    m2 = t[2].replace(root,"")
    if t[2][0:len(m2)] != m2:
        price3 = 1/price3

    elimit1 = elimit
    elimit2 = elimit1 * price1
    elimit3 = elimit2 * price2
    elimit = [elimit1, elimit2, elimit3]
    
    # Compute buy-sell direction
    bs_dir = []
    i = 0 
    # Step one
    ck1 = t[i][len(t[i])-3:len(t[i])]
    if ck1 == root:
        #buy
        bs_dir.append('asks')
    else:
        #sell
        bs_dir.append('bids')
    i = 1
    # Step two
    md1 = t[0].replace(root,'')
    ck2 = t[i][len(t[i])-3:len(t[i])]
    if ck2 == md1[len(md1)-3:len(md1)]:
        bs_dir.append('asks')
    else:
        bs_dir.append('bids')
    i = 2
    # Step three
    md2 = t[1].replace(md1,'')
    ck3 = t[i][len(t[i])-3:len(t[i])]
    if ck3 == root:
        #sell
        bs_dir.append('bids')
    else:
        #buy
        bs_dir.append('asks')

    for i in range(3):

        esum = 0.0
        ecounter = 0

        """ API Call """
        depth = client.get_order_book(symbol=t[i])
        time.sleep(.15)
        #namevalue = client.get_symbol_ticker(symbol=t[i])
        #time.sleep(.15)

        while esum < elimit[i]:
            try:
                eitem = depth[bs_dir[i]][ecounter]
            except:
                return 'error', 'error'
            price = float(eitem[0])
            esum = esum + price * float(eitem[1])
            ecounter = ecounter + 1
            
        #print(esum)
        #print("final price")
        #print(price)
        #print("indicator price")
        #print(namevalue['price'])
        price_dict2.update({t[i]:price})
        price_dict3.update({t[i]:{'price':price, 'quant':elimit[i]}})   
    new_gain = calc_gain(t,price_dict2)
    return new_gain, price_dict3

In [52]:
""" Define Functions """

def get_balance(root):
    
    """ API Call """
    balance = client.get_asset_balance(asset=root)
    time.sleep(.15)
    
    return balance

def buy(symbol, quant, price):
    
    """ API Call """
    order = client.order_limit_buy(symbol=symbol, quantity=quant, price=price)
    time.sleep(.5)
    
    """ API Call 
    open_orders = client.get_open_orders(symbol=symbol)
    if len(open_orders) == 0:
        return
    else:
        time.sleep(.25)
        return 
    """
    return

def sell(symbol, quant, price):
    
    """ API Call """
    order = client.order_limit_sell(symbol=symbol, quantity=quant, price=price)
    time.sleep(.5)
    
    return

def order(root, pd3, t, quant):
    
    i = 0 
    # Step one
    ck1 = t[i][len(t[i])-3:len(t[i])]
    if ck1 == root:
        print('buy')
        buy(t[i], pd3[t[i]]['quant'], pd3[t[i]]['price'])
    else:
        print('sell')
        sell(t[i], pd3[t[i]]['quant'], pd3[t[i]]['price'])
        
    i = 1
    # Step two
    md1 = t[0].replace(root,'')
    ck2 = t[i][len(t[i])-3:len(t[i])]
    if ck2 == md1[len(md1)-3:len(md1)]:
        print('buy')
        buy(t[i], full, pd3[t[i]]['price'])
    else:
        print('sell')
        sell(t[i], full, pd3[t[i]]['price'])

    i = 2
    # Step three
    md2 = t[1].replace(md1,'')
    ck3 = t[i][len(t[i])-3:len(t[i])]
    if ck3 == root:
        print('sell')
        sell(t[i], full, pd3[t[i]]['price'])
    else:
        print('buy')
        buy(t[i], full, pd3[t[i]]['price'])
        

In [62]:
# Call functions
'''
price_api() - get updated price list
get_trades(root, price_dict, symbols) - find triangular trade opportunities
calc_depth(root, t, elimit) - finds market depth adjusted opportunity 
get_balance(root) - return root balance
order(root, price_dict3, t, quant) - live triangle trading function

'''
start = time.time()

root = 'ETH'
trading_quant = .05
trading_quant_safety = .01
trading_min = 1.01


balance = {}

price_dict, symbols = price_api()
trades = get_trades(root, price_dict, symbols)

for i in range(1,10):
    iname = trades[i]['name']
    print(iname)
    print(trades[i]['gain'])
    
    adj_gain, price_dict3 = calc_depth(root, iname, trading_quant_safety)
    if adj_gain == 'error':
        print('Adjusted gain error')
    else:
        print("Adjusted gain:")
        print(adj_gain)
        print(price_dict3)
        if adj_gain > trading_min:
            print("***Initializing trade of " + str(iname))
            #order(root, price_dict3, iname, trading_quant)
            
end = time.time()
print("Machine time: " + str(end - start))


['BNBETH', 'WAVESBNB', 'WAVESETH']
0.03220724226304994
Adjusted gain:
0.9868438465331727
{'WAVESBNB': {'quant': 0.8277460475126232, 'price': 0.6939}, 'BNBETH': {'quant': 0.01, 'price': 0.012115}, 'WAVESETH': {'quant': 1.237658563864568, 'price': 0.008296}}
['BNBETH', 'RLCBNB', 'RLCETH']
0.030584806680320487
Adjusted gain:
0.9751119778415263
{'RLCETH': {'quant': 4.09775271045853, 'price': 0.002422}, 'BNBETH': {'quant': 0.01, 'price': 0.012115}, 'RLCBNB': {'quant': 0.8277460475126232, 'price': 0.20502}}
['BNBETH', 'XZCBNB', 'XZCETH']
0.02003535151423308
Adjusted gain:
0.9983653865905963
{'BNBETH': {'quant': 0.01, 'price': 0.012115}, 'XZCBNB': {'quant': 0.8277460475126232, 'price': 5.746}, 'XZCETH': {'quant': 0.14596121451465757, 'price': 0.069499}}
['TRIGETH', 'TRIGBNB', 'BNBETH']
0.011406218749999919
Adjusted gain:
0.9961559334811529
{'BNBETH': {'quant': 0.8371875000000001, 'price': 0.012079}, 'TRIGBNB': {'quant': 4.464285714285714, 'price': 0.18597}, 'TRIGETH': {'quant': 0.01, 'price':

In [61]:
for t in trades:
    print(trades[t])

{'name': ['BNBETH', 'WAVESBNB', 'WAVESETH'], 'gain': 0.029480367685949416}
{'name': ['ETHBTC', 'CDTBTC', 'CDTETH'], 'gain': 0.013213468259895444}
{'name': ['BNBETH', 'XZCBNB', 'XZCETH'], 'gain': 0.012915133797893219}
{'name': ['ETHBTC', 'CTRBTC', 'CTRETH'], 'gain': 0.012867299014522882}
{'name': ['BNBETH', 'OSTBNB', 'OSTETH'], 'gain': 0.010604481919362252}
{'name': ['ETHBTC', 'ICNBTC', 'ICNETH'], 'gain': 0.00992920983940171}
{'name': ['BNBETH', 'RLCBNB', 'RLCETH'], 'gain': 0.009879738077002598}
{'name': ['ETHBTC', 'BCDBTC', 'BCDETH'], 'gain': 0.009866117174037736}
{'name': ['BNBETH', 'BCPTBNB', 'BCPTETH'], 'gain': 0.009541106957106082}
{'name': ['ETHBTC', 'DNTBTC', 'DNTETH'], 'gain': 0.008746732336956553}


In [44]:
start = time.time()

end = time.time()
print("Machine time: " + str(end - start))

Machine time: 0.0
